In [ ]:
import time
from langchain.memory import ConversationBufferMemory


class TemporaryMemory:
    """Handles temporary chatbot memory and resets after inactivity."""

    def __init__(self, timeout=300):  # Default timeout: 5 minutes
        self.memory = ConversationBufferMemory()
        self.last_active = time.time()
        self.timeout = timeout

    def store_conversation(self, user_input, bot_response):
        """Stores conversation & updates last activity time."""
        self.memory.save_context({"input": user_input}, {
                                 "output": bot_response})
        self.last_active = time.time()

    def get_memory(self):
        """Retrieves conversation history."""
        return self.memory.load_memory_variables({})

    def reset_if_inactive(self):
        """Clears memory if inactive for too long."""
        if time.time() - self.last_active > self.timeout:
            self.memory.clear()
            return "Memory reset due to inactivity."
        return self.memory.load_memory_variables({})


chat_memory = TemporaryMemory(timeout=120)  # 2-minute reset

chat_memory.store_conversation("What is 2+2?", "4")
chat_memory.store_conversation("Who discovered gravity?", "Isaac Newton")

print(chat_memory.get_memory())  # Retrieves stored conversations

time.sleep(130)  # Simulating user inactivity for 2 minutes
print(chat_memory.reset_if_inactive())  # Resets memory automatically

C:\Users\oulda\AppData\Local\Temp\ipykernel_884\1051123107.py:9: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  self.memory = ConversationBufferMemory()


{'history': 'Human: What is 2+2?\nAI: 4\nHuman: Who discovered gravity?\nAI: Isaac Newton'}


In [3]:
from flask import Flask, request, jsonify, render_template, send_from_directory
import random
import json
import google.generativeai as genai
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

app = Flask(__name__)

# Google Gemini API Key
GEMINI_API_QUIZZ_KEY = os.getenv("GEMINI_API_QUIZZ_KEY")
# Configure la bibliothèque avec votre clé API Google
genai.configure(api_key=GEMINI_API_QUIZZ_KEY)

# Banque de questions de secours (fallback) au cas où l'appel à l'API échouerait
fallback_question_bank = {
    "maths": [
        {
            "question": "Quel est le théorème de Pythagore et comment s'applique-t-il dans un triangle rectangle ?",
            "options": ["a² + b² = c²", "a + b = c", "a² - b² = c²", "a * b = c"],
            "correct": "a² + b² = c²"
        },
        {
            "question": "Comment résoudre une équation quadratique ?",
            "options": ["Par factorisation", "Par complétion du carré", "Par la formule quadratique", "Toutes les réponses"],
            "correct": "Toutes les réponses"
        }
    ],
    "science": [
        {
            "question": "Quelles sont les lois de la thermodynamique et leur importance dans la physique ?",
            "options": ["Première loi", "Deuxième loi", "Troisième loi", "Quatrième loi"],
            "correct": "Première loi"
        }
    ],
    "biologie": [
        {
            "question": "Comment se déroule la photosynthèse chez les plantes ?",
            "options": ["Conversion de la lumière en énergie chimique", "Production d'oxygène", "Utilisation du dioxyde de carbone", "Synthèse des protéines"],
            "correct": "Conversion de la lumière en énergie chimique"
        }
    ],
    "agroalimentaire": [
        {
            "question": "Quel est le rôle de la traçabilité dans la sécurité alimentaire ?",
            "options": ["Assurer la qualité", "Garantir la provenance", "Réduire les coûts", "Améliorer le goût"],
            "correct": "Garantir la provenance"
        }
    ],
    "informatique": [
        {
            "question": "Qu'est-ce que le cloud computing et comment révolutionne-t-il l'informatique ?",
            "options": ["Stockage local", "Informatique en nuage", "Réseaux privés", "Serveurs dédiés"],
            "correct": "Informatique en nuage"
        }
    ]
}


def generate_question_with_gemini(category):
    prompt = (
        "Crée une question de quiz en français pour des étudiants en maths. "
        "La question doit avoir 4 options de réponses et indiquer la bonne réponse. "
        "Retourne le résultat sous forme de JSON avec les clés 'question', 'options' (liste de 4 éléments) et 'correct'."
    )


    try:
        model = genai.GenerativeModel("gemini-1.5-pro-latest")
        response = model.generate_content(prompt)
        print("Raw Gemini response:", response.text)

        # Vérifier si response.text n'est pas vide
        if not response.text.strip():
            raise ValueError("Réponse vide")
        result = json.loads(response.text)
        return result
    except Exception as e:
        print(f"Erreur lors de l'appel à Google Gemini: {e}")
        return random.choice(fallback_question_bank.get(category, []))


@app.route("/")
def home():
    return render_template("index.html")


@app.route("/quiz", methods=["GET"])
def get_quiz():
    # La catégorie peut être spécifiée en paramètre de requête, par exemple ?category=maths
    category = request.args.get("category", "maths")
    question = generate_question_with_gemini(category)
    return jsonify(question)


@app.route("/answer", methods=["POST"])
def submit_answer():
    data = request.json
    question_text = data.get("question")
    user_answer = data.get("answer")
    correct_answer = data.get("correct")

    if not correct_answer:
        return jsonify({"error": "La bonne réponse n'est pas fournie"}), 404

    if user_answer == correct_answer:
        return jsonify({"result": "correct", "correct_answer": correct_answer})
    else:
        return jsonify({"result": "incorrect", "correct_answer": correct_answer})

@app.route("/favicon.ico")
def favicon():
    return send_from_directory("static", "favicon.ico")


if __name__ == "__main__":
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [03/Mar/2025 22:04:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2025 22:04:40] "GET /quiz?category=maths HTTP/1.1" 200 -


Raw Gemini response: ```json
{
  "question": "Quel est le résultat de l'équation suivante : √(16) + 3 * 2 - 5?",
  "options": [
    "3",
    "5",
    "7",
    "9"
  ],
  "correct": "5"
}
```

Erreur lors de l'appel à Google Gemini: Expecting value: line 1 column 1 (char 0)
